In [1]:
##############################################################
# CLTV Prediction with BG-NBD and Gamma-Gamma (Online Retail II)
##############################################################

# 1. Library Installation (Required for Kaggle/Colab environments)
!pip install lifetimes

import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

# Display Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)

###############################################################
# TASK 1: Data Preparation
###############################################################

# Dataset path on Kaggle
file_path = "/kaggle/input/datasets/lakshmi25npathi/online-retail-dataset/online_retail_II.xlsx"

# Reading the Excel file (Sheet: Year 2010-2011)
df_ = pd.read_excel(file_path, sheet_name="Year 2010-2011")
df = df_.copy()

# Functions for handling outliers
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit, 0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit, 0)

# Data Preprocessing Steps
df.dropna(inplace=True)
df = df[~df["Invoice"].astype(str).str.contains("C", na=False)]  # Remove cancellations
df = df[df["Quantity"] > 0]
df = df[df["Price"] > 0]

# Filtering for United Kingdom customers only (As per project requirement)
df = df[df["Country"] == "United Kingdom"]

# Suppressing Outliers
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

df["TotalPrice"] = df["Quantity"] * df["Price"]
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
today_date = dt.datetime(2011, 12, 11)

###############################################################
# TASK 2: Creating the CLTV Data Structure (RFM Metrics)
###############################################################

# Recency: Time between first and last purchase (in days)
# T (Tenure): Age of the customer (Analysis date - First purchase date in days)
# Frequency: Total number of repeat purchases (Transaction count)
# Monetary: Total price
cltv_df = df.groupby('Customer ID').agg({
    'InvoiceDate': [lambda date: (date.max() - date.min()).days,
                    lambda date: (today_date - date.min()).days],
    'Invoice': lambda num: num.nunique(),
    'TotalPrice': lambda total_price: total_price.sum()
})

cltv_df.columns = cltv_df.columns.droplevel(0)
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

# Monetary value redefined as average earnings per purchase
cltv_df["monetary"] = cltv_df["monetary"] / cltv_df["frequency"]

# Conversion of recency and T to weekly units
cltv_df["recency"] = cltv_df["recency"] / 7
cltv_df["T"] = cltv_df["T"] / 7

# BG-NBD model requires repeat customers (frequency > 1)
cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

###############################################################
# TASK 3: Model Fitting and CLTV Prediction
###############################################################

# Establishing the BG/NBD Model (Predicts expected number of transactions)
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'], cltv_df['recency'], cltv_df['T'])

# Establishing the Gamma-Gamma Model (Predicts expected average profitability)
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df['frequency'], cltv_df['monetary'])

# PDF Task 1: 6-Month CLTV Prediction
cltv_df["cltv_6_month"] = ggf.customer_lifetime_value(bgf,
                                                       cltv_df['frequency'],
                                                       cltv_df['recency'],
                                                       cltv_df['T'],
                                                       cltv_df['monetary'],
                                                       time=6,  # 6 months
                                                       freq="W",  # weekly
                                                       discount_rate=0.01)

# PDF Task 2: 1-Month and 12-Month CLTV Predictions
cltv_df["cltv_1_month"] = ggf.customer_lifetime_value(bgf, cltv_df['frequency'], cltv_df['recency'], cltv_df['T'], cltv_df['monetary'], time=1, freq="W", discount_rate=0.01)
cltv_df["cltv_12_month"] = ggf.customer_lifetime_value(bgf, cltv_df['frequency'], cltv_df['recency'], cltv_df['T'], cltv_df['monetary'], time=12, freq="W", discount_rate=0.01)

###############################################################
# TASK 4: Segmentation
###############################################################

# Dividing customers into 4 segments based on 6-month CLTV
cltv_df["segment"] = pd.qcut(cltv_df["cltv_6_month"], 4, labels=["D", "C", "B", "A"])

# Displaying Results
print("--- UK Customers: Top 10 by 6-Month CLTV ---")
print(cltv_df.sort_values("cltv_6_month", ascending=False).head(10))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 8.4 MB/s eta 0:00:00
--- UK Customers: Top 10 by 6-Month CLTV ---
             recency       T  frequency  monetary  cltv_6_month  cltv_1_month  cltv_12_month segment
Customer ID                                                                                         
18102.0000   52.2857 52.5714         60 3582.7037    85596.3189    14875.4326    163487.0583       A
14096.0000   13.8571 14.5714         17 3159.4912    55654.1274     9856.4348    104907.4989       A
17450.0000   51.2857 52.5714         46 2627.0337    48485.8846     8426.5013     92603.9163       A
17511.0000   52.8571 53.4286         31 2921.8200    36794.3729     6393.8510     70280.7843       A
16684.0000   50.4286 51.2857         28 2118.9150    25068.9600     4358.5975     47863.6232       A
13694.0000   52.7143 53.4286         50 1267.2430    25057.6376     4353.9241     47866.1460       A
14088.0000   44.5714 46.1429         13 3860.1431    25012.9687    